In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import pacf 
from statsmodels.tsa.stattools import acf
import numpy as np

### Plotting the autocorrelation and partial autocorrelation plots to find the number of lags in the time series that we will use to validate the time series model and finally build the testing case
Autocorrelation and partial autocorrelation plots are heavily used in time series analysis and forecasting.These are plots that graphically summarize the strength of a relationship with an observation in a time series with observations at prior time steps.

We can calculate the correlation for time series observations with observations with previous time steps, called lags. Because the correlation of the time series observations is calculated with values of the same series at previous times, this is called a serial correlation, or an autocorrelation.

A plot of the autocorrelation of a time series by lag is called the AutoCorrelation Function, or the acronym ACF. This plot is sometimes called a correlogram or an autocorrelation plot.


In [ ]:
df = pd.read_pickle('Updated_CovidDataset.pkl')
#values = series.values
## Only values from 
from datetime import datetime
df['INDEX_DATE'] = df['INDEX_DATE'].astype('datetime64[ns]')
datetime_str = '11-30-20'
datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
df_validation = df[df['INDEX_DATE'] <= datetime_object ].reset_index(drop= True)
data =df_validation['D1_COVID_NEW_ADM_CNT'].values
# # data =df[Feature_set + ['D1_COVID_NEW_ADM_CNT']].values
# data = data.reshape(-1,1)

# series = read_csv('daily-minimum-temperatures.csv', header=0, index_col=0)

plot_acf(data, lags=5)
pyplot.xlabel('lags')
pyplot.ylabel('Autocorrelation')

pyplot.show()

A partial autocorrelation is a summary of the relationship between an observation in a time series with observations at prior time steps with the relationships of intervening observations removed.

The blue shadow within the plot showing the confidence interval is relative and signifies which one is more statistically significant than the other.

- stattools.pacf computes the confidence interval around the estimated pacf, i.e. it's centered at the actual value
- graphics.tsa.plot_pacf takes that confidence interval and subtracts the estimated pacf, So the confidence interval is centered at zero.

In [ ]:
plot_pacf(data, lags=5, alpha=.05)
pyplot.xlabel('lags')
pyplot.ylabel('Partial Autocorrelation')
pyplot.show()


In [ ]:
A, B =pacf(data, nlags=5, alpha=.05)


Why are we doing with PACF: In practical application of AR modelling, it is important to know how much information from the past (lag) we can reliably use to convert the time series into a supervised learning model AND how often in the future it should run (like every day, every fifth day etc.) to update the model. Since we are dealing with a NON-STATIONARY signal and we are not sure of the trend and seasonality, we cannot use a fixed model to run indefinitely.

Looking at the plot of partial autocorrelation, we can see that a lag value of 3 can be used for training the model to start with i.e. 3 past values from the past with the current value can be used to build the training data set for the supervised learning model. 

#### Designing the walk forward validation 
rolling forward corss validation to assess the behaviors of the different models


In [ ]:
# forecast monthly births with random forest
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor
from matplotlib import pyplot
import pandas as pd
import xgboost as xgb
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = (1 if type(data) is list else data.shape[1])
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

scaler = StandardScaler()
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]

# fit an random forest model and make a one step prediction
def random_forest_forecast(train, testX):
    # transform list into array
    train = asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    
    # fit model
    model = RandomForestRegressor(n_estimators=1000)
#     model = GradientBoostingRegressor(n_estimators=250, max_depth=3,
#                                 learning_rate=.1, min_samples_leaf=9,
#                                 min_samples_split=9)
#     #model = LinearRegression()
#     trainX = scaler.fit_transform(trainX)
# #     testX = testX.reshape(-1,1)
#     print(testX)
#     testX = scaler.transform([testX])
    model.fit(trainX, trainy)
    # make a one-step prediction
    
    yhat = model.predict([testX])
    return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    l, test = train_test_split(data, n_test)
#     print(train)
#     print(test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
        yhat = np.ceil(yhat)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
#         print(test[i])
        history.append(test[i])
        history = history[1:]
        print(len(history))
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, -1], predictions

df = pd.read_pickle('Updated_CovidDataset.pkl')
#values = series.values
## Only values from 
from datetime import datetime
df['INDEX_DATE'] = df['INDEX_DATE'].astype('datetime64[ns]')
datetime_str = '11-30-20'
datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
df_validation = df[df['INDEX_DATE'] <= datetime_object ].reset_index(drop= True)
data =df_validation['D1_COVID_NEW_ADM_CNT'].values
data = data.reshape(-1,1)
# transform the time series data into supervised learning

data = series_to_supervised(data, n_in=2)
# Add the other features which may be useful for prediction
# evaluate
mae, y, yhat = walk_forward_validation(data, 30)
yhat =np.ceil(np.array(yhat))
print('MAE: %.3f' % mae)
# plot expected vs predicted
pyplot.figure(figsize=(10,8))
pyplot.plot(y, '--ob',label='Expected')
pyplot.plot(yhat, '-xr',label='Predicted')
pyplot.legend()
pyplot.grid(True)
pyplot.show()

In [ ]:
data


### Deployment scenario that is realistic
Every day mid night we receive the number from the last day and use that to retrain the new model and predict the next day numbers
### What kind of penalty or performance metrices are we looking at?
- Penalty for underprediction ?
- No penalty for over prediction ?


### Simulating and testing a deployment scenario
Now that we have validated our model till 11-30-2020, we are going to use the model to predict the next day but use the same rolling walk forward method, where we use the past data to predict the future data

In [ ]:
# from datetime import datetime
from datetime import datetime as dt, timedelta
df = pd.read_pickle('Updated_CovidDataset.pkl')
df['INDEX_DATE'] = df['INDEX_DATE'].astype('datetime64[ns]')
datetime_str_current = '12-1-20'
date_back =3

datetime_object = dt.strptime(datetime_str, '%m-%d-%y')
df_test = df[df['INDEX_DATE'] >= datetime_object ].reset_index(drop= True)
data_test =df_test['D1_COVID_NEW_ADM_CNT'].values
# # data =df[Feature_set + ['D1_COVID_NEW_ADM_CNT']].values
#data = data.reshape(-1,1)

# # transform the time series data into supervised learning
## ------------------------------------------------
datetime_str = '11-30-20'
datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
df_validation = df[df['INDEX_DATE'] <= datetime_object ].reset_index(drop= True)
data =df_validation['D2_COVID_NEW_ADM_CNT'].values
#data =(df_validation['COVID_MED_SURG_NO_HFNC_PCT_M'].values)/100

data = data.reshape(-1,1)

train_current = series_to_supervised(data, n_in=4)
# split into input and output columns
trainX, trainy = train_current[:, :-1], train_current[:, -1]
# # fit model
model = GradientBoostingRegressor(n_estimators=250, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)
model.fit(trainX, trainy)
##------------------------------------------------------
# # construct an input for a new prediction
# row = data[-3:].flatten()
row  = data[-4:].flatten()

# # make a one-step prediction
yhat = model.predict(asarray([row]))
print('Input: %s, Predicted: %.3f' % (row, yhat[0]))

In [ ]:
import datetime
from datetime import datetime as dt, timedelta
df = pd.read_pickle('Updated_CovidDataset.pkl')
df['INDEX_DATE'] = df['INDEX_DATE'].astype('datetime64[ns]')
datetime_str_current = '12-1-20'

datetime_object = dt.strptime(datetime_str, '%m-%d-%y')

In [ ]:
import datetime
from datetime import datetime as dt, timedelta
days = datetime.timedelta(5)

new_date = datetime_object - days
new_date